In [1]:
import os
import sys
from quetzal.os.parallel_call import parallel_call_notebook, parallel_call_notebooks

In [2]:
if not os.path.exists('log/'):
    os.makedirs('log/')

# Launcher

This notebook automatically launches all operations for a complete model run. One can decide to skip network preparation steps, as they take multiple hours for the region of Germany and the networks are readily included in the repository.

Detailed explainations of certain steps can be found in the corresponding notebook. All ASSUMPTIONS are gathered in the `parameters.xls` file. This file also includes scenarios.

In [3]:
kwargs = {'workers':2, 'errout_suffix':True, 'sleep':1,'stdout_path':r'log/out.txt', 'stderr_path':r'log/err.txt'}

In [4]:
scenarios = ['base']

In [5]:
import time
start = time.time()

# Network preparation

Execute all `prep1` and `prep2` steps.

In [ ]:
# Network generation
parallel_call_notebook('prep10_zones.ipynb', arg_list=scenarios, **kwargs)
parallel_call_notebooks(
    ('prep11_road.ipynb', scenarios),
    ('prep12_rail_coach.ipynb', scenarios),
    ('prep14_bus.ipynb', scenarios),
    **kwargs)
parallel_call_notebook('prep13_air.ipynb', arg_list=scenarios, **kwargs)

In [ ]:
# Access/egress distances to PT services based on networks and census data
parallel_call_notebook('prep15_census_distances.ipynb', arg_list=scenarios, **kwargs)

In [ ]:
# Network aggregation
# Either cluster short-distance PT
#parallel_call_notebook('prep20_cluster.ipynb', arg_list=scenarios, **kwargs)
#parallel_call_notebook('prep21_access_egress.ipynb', arg_list=scenarios, **kwargs)
# Or aggregate it
parallel_call_notebook('prep20_aggregate_pt.ipynb', arg_list=scenarios, **kwargs)
parallel_call_notebook('prep21_access_egress_road.ipynb', arg_list=scenarios, **kwargs)

# Pathfinders and level of service

In [ ]:
# Pathfinders
parallel_call_notebooks(
    ('prep30_pathfinder_pt.ipynb', scenarios),
    ('prep30_pathfinder_road.ipynb', scenarios),
    **kwargs)

In [ ]:
# PT LoS attributes
parallel_call_notebook('prep30_pathfinder_pt.ipynb', arg_list=scenarios, **kwargs)
parallel_call_notebook('prep31_prices_pt.ipynb', arg_list=scenarios, **kwargs)
parallel_call_notebook('prep33_non-motor.ipynb', arg_list=scenarios, **kwargs)

# Estimation

In [7]:
# Calibration data preparation
# NOTE: only applicapbe if you have access to the calibration dataset
parallel_call_notebook('cal10_input_data.ipynb', arg_list=scenarios, **kwargs)
parallel_call_notebook('cal11_los_columns.ipynb', arg_list=scenarios, **kwargs)
#parallel_call_notebook('cal1x_validation_data.ipynb', arg_list=scenarios, **kwargs)
#parallel_call_notebook('cal19_vot.ipynb', arg_list=scenarios, **kwargs)

0 cal11_los_columns.py base
subprocess **cal11_los_columns.py0 base** terminated with an error.
184 seconds


In [ ]:
# Estimation
parallel_call_notebook('cal20_estimation.ipynb', arg_list=scenarios, **kwargs)

# Modelling

Except for the logit model step, all modelling steps require access to the OD volumes dataset.

In [ ]:
# Generation of the OD matrix
parallel_call_notebook('model_volumes.ipynb', arg_list=scenarios, **kwargs)

In [ ]:
# Mode choice probabilities
parallel_call_notebook('model_logit.ipynb', arg_list=scenarios, **kwargs)

In [ ]:
# Assignment
parallel_call_notebooks(
    ('model_assignment.ipynb', scenarios),
    ('model_assignment_inner-zonal.ipynb', scenarios),
    **kwargs)

In [ ]:
# Post-processing
parallel_call_notebook('model_emissions.ipynb', arg_list=scenarios, **kwargs)

In [ ]:
# Total execution time
end = time.time()
int(end - start)